<a href="https://colab.research.google.com/github/anshulrath0re/MNIST_Digit_classifier/blob/main/MNIST_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image

# Data Loading

In [ ]:
def load_images(file_path):
  with open(file_path, 'rb') as f:
    data = f.read()
  binary_data = np.frombuffer(data, dtype= np.uint8)[16:]
  row = int.from_bytes(data[8:12], byteorder='big', signed = False)
  column = int.from_bytes(data[12:16], byteorder= 'big' , signed= False)
  images = binary_data.reshape((-1, row, column,))
  return images

In [ ]:
images = load_images("/content/train-images.idx3-ubyte")

In [ ]:
def load_labels(file_path):
  with open(file_path, 'rb') as f:
    data = f.read()
  labels = np.frombuffer(data, dtype= np.uint8)[8:]
  return labels

In [ ]:
labels = load_labels('/content/train-labels.idx1-ubyte')

In [ ]:
print(labels)

[5 0 4 ... 5 6 8]


In [ ]:
import torch
import numpy as np
from PIL import Image
from IPython.display import display
from google.colab import files

def tensor_to_png(tensor: torch.Tensor, output_path: str) -> None:
    """
    Convert a 1D torch.Tensor of length 784 into a 28×28 grayscale PNG,
    display it inline in Colab, and offer it for download.
    """
    # 1) Validate shape
    if tensor.numel() != 784:
        raise ValueError(f"Expected tensor of 784 elements, got {tensor.numel()}")

    # 2) Move to CPU & numpy, reshape to 2D
    arr = tensor.detach().cpu().numpy().reshape(28, 28)

    # 3) Normalize / scale to uint8
    arr_min, arr_max = arr.min(), arr.max()
    if arr_max == arr_min:
        img_data = np.full(arr.shape, 128, dtype=np.uint8)
    else:
        scaled   = (arr - arr_min) / (arr_max - arr_min)
        img_data = (scaled * 255).astype(np.uint8)

    # 4) Create PIL image
    img = Image.fromarray(img_data, mode='L')
    img.save(output_path)
    print(f"Saved to {output_path!r}")

    # 5) Display inline
    display(img)

    # 6) Download link
    files.download(output_path)


# --- Example usage ---
# Suppose `X` is your (60000, 784) tensor and you want the 2nd image:
print(tensor_to_png(flatten_tensor_features[2], "mnist_sample.png"))



Saved to 'mnist_sample.png'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

None


# Normalizing Data

In [ ]:
normalized_images = images/256

In [ ]:
normalized_images.shape

(60000, 28, 28)

In [ ]:
flatten_x = normalized_images.reshape(-1, 784)

In [ ]:
for i in range(28):
 print(flatten_x[0][i*28 : i*28+28])
 print('\n')


In [ ]:
flatten_x[0].shape

(784,)

In [ ]:
encoder = OneHotEncoder(sparse_output= False)
rl =labels.reshape(-1,1)
encoder.fit(rl)
one_hot_labels = encoder.transform(rl)

In [ ]:
print(one_hot_labels[1])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# no need of normalizing labels as they should be one hot encoded
# normalized_labels = labels/10

# Trying Linear recurrsion


In [ ]:
#MSE
def MSE(w_mse, X_mse, y_mse):
  sample_size = X_mse.shape[0]
  y__ = X_mse.dot(w_mse)
  error_sq = (y__ - y_mse)**2
  return np.mean(error_sq)/2

In [ ]:
w = np.array(np.array(float))

In [ ]:
w

array(<class 'float'>, dtype=object)

In [ ]:
def lr(X_train, y_train):
  for i in range(y_train.shape[0]):
    w[i] =np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train

In [ ]:
import numpy as np

def lr_closed_form(X, Y):
    # X: (n_samples, n_features)
    # Y: (n_samples, n_targets)   (for single-vector y, n_targets = 1)
    # use pseudo-inverse for numeric stability / singular X^T X
    XtX = X.T @ X
    XtX_inv = np.linalg.pinv(XtX)       # shape (n_features, n_features)
    W = XtX_inv @ (X.T @ Y)             # shape (n_features, n_targets)
    return W

# Example usage:
W = lr_closed_form(flatten_x, one_hot_labels)
# If you just have Y as a vector (shape (n_samples,)), reshape it:
# W = lr_closed_form(flatten_x, y_vector.reshape(-1, 1))


In [ ]:
print(labels[10])

3


In [ ]:
print(flatten_x[10].dot(W))

[ 0.09917694  0.19435205  0.09386254  0.67154284 -0.1785025   0.04406953
 -0.14383908  0.07933444  0.07603549  0.24246847]


In [ ]:
MSE(W, flatten_x, one_hot_labels)

np.float64(0.019550186200288516)

In [ ]:
test_labels = load_labels('/content/t10k-labels(1).idx1-ubyte')

checking accuracy

In [ ]:
import numpy as np

def compute_accuracy(X, W, labels):
    """
    X      : array of shape (n_samples, n_features)
    W      : weight matrix of shape (n_features, n_classes)
    labels : integer array of shape (n_samples,), values in [0..n_classes-1]
    """
    # Compute the raw scores for each class
    scores = X @ W                       # shape (n_samples, n_classes)
    # Pick the class with the highest score
    preds = np.argmax(scores, axis=1)    # shape (n_samples,)
    # Compare to the true labels
    correct = (preds == labels)          # boolean array
    # Return fraction correct
    return correct.mean()



In [ ]:
acc = compute_accuracy(flatten_x, W, labels)
print(f"Model accuracy: {acc*100:.2f}%")

Model accuracy: 85.19%


In [ ]:
from PIL import Image
import numpy as np

def predict_from_png(path: str, W: np.ndarray) -> int:
    """
    path: filesystem path to a 28×28 grayscale PNG
    W   : weight matrix of shape (n_features, n_classes), here (784, 10)

    Returns the predicted digit (0–9).
    """
    # 1. Load and ensure grayscale
    img = Image.open(path).convert('L')   # 'L' = 8-bit grayscale

    # 2. Verify size (optional)
    if img.size != (28, 28):
        raise ValueError(f"Expected 28×28 image, got {img.size}")

    # 3. To NumPy array and normalize
    arr = np.asarray(img, dtype=np.float32) / 255.0  # shape (28,28)

    # 4. Flatten to (784,)
    x = arr.ravel()  # or arr.flatten()

    # 5. Compute scores and pick max
    scores = x @ W   # shape (10,)
    pred  = int(np.argmax(scores))

    return pred

# Example usage:
# >>> digit = predict_from_png("some_digit.png", W)
# >>> print("I think it’s a", digit)


In [ ]:
print(predict_from_png("/content/custom_sample.png",W))

9


test data accuracy

In [ ]:
acc = compute_accuracy(test_images, W, test_labels)
print(f"Model accuracy: {acc*100:.2f}%")

Model accuracy: 85.34%


In [ ]:
test_images = (load_images("/content/t10k-images.idx3-ubyte").reshape(-1, 784))/256

In [ ]:
import numpy as np

def per_class_accuracy(X, W, labels, classes=None):

    scores = X @ W                          # shape (n_samples, n_classes)
    preds  = np.argmax(scores, axis=1)      # shape (n_samples,)

    # 2) decide which classes to report
    if classes is None:
        classes = np.unique(labels)
    per_class_acc = {}

    for cls in classes:
        # mask of all samples whose true label == cls
        idx = np.where(labels == cls)[0]
        if idx.size == 0:
            per_class_acc[cls] = np.nan    # no samples of this class
        else:
            # compare preds[idx] vs cls
            per_class_acc[cls] = np.mean(preds[idx] == cls)

    return per_class_acc

In [ ]:
#train data Accuracy per digit
classes = list(range(1, 10))
accs = per_class_accuracy(flatten_x, W, labels, classes)
for digit, acc in accs.items():
    print(f"Digit {digit}: {acc*100:.2f}%")

Digit 1: 96.62%
Digit 2: 80.60%
Digit 3: 84.91%
Digit 4: 87.73%
Digit 5: 69.84%
Digit 6: 92.78%
Digit 7: 86.05%
Digit 8: 74.48%
Digit 9: 80.32%


In [ ]:
#test data Accuracy per digit
classes = list(range(1, 10))
accs = per_class_accuracy(test_images, W, test_labels, classes)
for digit, acc in accs.items():
    print(f"Digit {digit}: {acc*100:.2f}%")

Digit 1: 97.53%
Digit 2: 78.39%
Digit 3: 87.82%
Digit 4: 88.80%
Digit 5: 69.96%
Digit 6: 91.02%
Digit 7: 85.31%
Digit 8: 76.28%
Digit 9: 79.39%


In [ ]:
mean_digit_image(labels, images , num= 3, save_path='/content/custom_sample.png')

In [ ]:
def mean_digit_image(labels: np.ndarray,
                     images: np.ndarray,
                     num: int,
                     normalized_images: np.ndarray = None,
                     save_path: str = None) -> Image.Image:

    # Select which array to average
    if normalized_images is not None:
        data = normalized_images  # floats in [0, 1]
        scale = 255.0
    else:
        # cast to float to avoid uint8 overflow
        data = images.astype(np.float64)
        scale = 1.0

    # Mask for samples of the desired digit
    mask = (labels == num)
    if not np.any(mask):
        raise ValueError(f"No examples found for digit {num}")

    # Compute the mean over axis=0 → shape (28, 28)
    mean_mat = data[mask].mean(axis=0) * scale

    # Clip to valid byte range and convert to uint8
    mean_mat = np.clip(mean_mat, 0, 255).astype(np.uint8)

    # Build PIL image
    img = Image.fromarray(mean_mat, mode='L')

    # Optionally write to disk
    if save_path:
        img.save(save_path, format='PNG')

    return img

# Dataset and dataloader configuration


In [ ]:
tensor_features = torch.from_numpy(normalized_images)

cross check here if features are flattening in the right way

In [ ]:
tensor_features.shape[0]

60000

In [ ]:
flatten_tensor_features = tensor_features.reshape(tensor_features.shape[0], tensor_features.shape[1]*tensor_features.shape[2])

In [ ]:
print(tensor_features.shape)

torch.Size([60000, 28, 28])


In [ ]:
print(flatten_tensor_features.shape[0])

60000


In [ ]:
tensor_labels = torch.from_numpy(labels)

/tmp/ipython-input-23-1508344864.py:1: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  tensor_labels = torch.from_numpy(labels)


In [ ]:
tensor_labels

tensor([5, 0, 4,  ..., 5, 6, 8], dtype=torch.uint8)

In [ ]:
print(tensor_labels[0])

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=torch.float64)


In [ ]:
dataset = TensorDataset( flatten_tensor_features , tensor_labels)

In [ ]:
dataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [ ]:
loader = DataLoader( dataset, batch_size = 50, shuffle = False)

# NN Design

In [ ]:
model = nn.Sequential(
    nn.Linear(784, 128, dtype=float),
    nn.ReLU(True),
    nn.Linear(128,64, dtype=float),
    nn.ReLU(True),
    nn.Linear(64, 10, dtype = float),
)

In [ ]:
model

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU(inplace=True)
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU(inplace=True)
  (4): Linear(in_features=64, out_features=10, bias=True)
)

# Loss Fuction and Optimizer

In [ ]:
criterian = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model.parameters(),lr = 0.001 )

# Training Loop

In [ ]:
softmax_layer = nn.Softmax(dim=0)

In [ ]:
flatten_tensor_features[0].shape

torch.Size([784])

In [ ]:
tensor_labels[500]

tensor(3, dtype=torch.uint8)

In [ ]:
running_loss = 0

In [ ]:
for epoch in range(5):
  running_loss = 0
  for X_batch, y_batch in loader:
    #forward bass
    y_pridict = model(X_batch)
    loss = criterian(y_pridict , y_batch)
    #back propogate
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

  print(f"Epoch {epoch+1}, Training loss: {running_loss / len(loader):.4f}")

Epoch 1, Training loss: 0.3357
Epoch 2, Training loss: 0.1426
Epoch 3, Training loss: 0.0954
Epoch 4, Training loss: 0.0687
Epoch 5, Training loss: 0.0515


In [ ]:
print(model(flatten_tensor_features[0]))

tensor([5.0171e-06, 8.0471e-16, 6.2966e-09, 3.2412e-03, 3.7278e-17, 9.9675e-01,
        2.5862e-13, 5.2315e-07, 1.5972e-08, 1.3467e-12], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [ ]:
u = softmax_layer(model(flatten_tensor_features[3]))

In [ ]:
print(max(u.argmax(dim=0)))

TypeError: iteration over a 0-d tensor